# Fit a simple poisson/gaussian model to IPNO test bench data

calin/examples/calib/ipno spe fits poisson gaussian.ipynb - Stephen Fegan - 2017-04-21

Copyright 2017, Stephen Fegan <sfegan@llr.in2p3.fr>
LLR, Ecole polytechnique, CNRS/IN2P3, Universite Paris-Saclay

This file is part of "__calin__". "__calin__" is free software: you can redistribute it 
and/or modify it under the terms of the GNU General Public License version 2 or later, 
as published by the Free Software Foundation. "__calin__" is distributed in the hope 
that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty 
of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public 
License for more details.

## Introduction

This notebook loads IPNO single photo-electron test-bench data and fits it using a Poisson/Gaussian model.

First, import the required packages.

In [ ]:
%pylab inline
import calin.diagnostics.functional
import calin.io.sql_transceiver
import calin.calib.spe_fit
import calin.math.histogram
import calin.math.optimizer
import calin.math.pdf_1d
import calin.iact_data.ipno
import calin.plotting
import calin.math.data_modeling

## Load the IPNO SPE and pedestal data files as histograms

Apply cuts on the minimum value to filter out ADC values of zero.

In [ ]:
mes_hist = calin.iact_data.ipno.make_ipno_adc_hist('/CTA/IPNO/2017-04-06-B/SET_1/5_LEDs_8.5V.bin', value_min=10)
ped_hist = calin.iact_data.ipno.make_ipno_adc_hist('/CTA/IPNO/2017-04-06-B/SET_1/pedestal_3.bin', value_min=10)

## Plot the histograms on linear and logarithmic scales

In [ ]:
calin.plotting.plot_histogram(mes_hist,label="SPE")
calin.plotting.plot_histogram(ped_hist,label="Ped")
legend()
xlabel('Charge [ADC samples]')
ylabel('Number of events')

In [ ]:
calin.plotting.plot_histogram(mes_hist,label="SPE")
calin.plotting.plot_histogram(ped_hist,label="Ped")
legend()
xlabel('Charge [ADC samples]')
ylabel('Number of events')
gca().set_yscale('log')

## Set up the model and cost function

There are four components:
1. the model for the pedestal, a binned Gaussian with bin width derived from histogram
2. the model of the single electron spectrum (SES), a binned Gaussian cut off at negative values, since the charge from the PMT cannot be negative
3. model for the multi electron spectrum (MES) that combines the pedestal and SES models with a Poisson model for the number of PEs, calculating the resultant MES shape through convolutions.
4. the cost function that computes the (negative of the) log likelihood of the measured MES and pedestal runs.

The configuration of the MES model is somewhat tricky as the bounds of the FFT must be specified. Here the left bound of the FFT is taken to be the lowest bin from the MES and pedestal histopgrams. Thenumber of bins in th FFT is basically twice the number in the MES histogram adjusted for the left bound. This may be automatized in future versions.

In [ ]:
ped_gauss_pdf = calin.math.pdf_1d.BinnedGaussianPDF(mes_hist.dxval())

In [ ]:
ses_g_pdf = calin.math.pdf_1d.LimitedGaussianPDF(0,numpy.inf,mes_hist.dxval())

In [ ]:
xleft = min(mes_hist.xval_left(0), ped_hist.xval_left(0))
#xleft = mes_hist.xval_left(0)

mes_model_g = calin.calib.spe_fit.GeneralPoissonMES(xleft, mes_hist.dxval(),\
        mes_hist.size()*2 + int((mes_hist.xval_left(0)-xleft)/mes_hist.dxval()), \
        ses_g_pdf, ped_gauss_pdf)

In [ ]:
# Uncomment first version to ONLY fit MES. Uncomment second version to fit both MES and Ped
#like_g = calin.calib.spe_fit.SPELikelihood(mes_model_g, mes_hist)
like_g = calin.calib.spe_fit.SPELikelihood(mes_model_g, mes_hist, ped_hist)

## Print list of cost function axes with intrinsic limits

In [ ]:
for iax, ax in enumerate(like_g.domain_axes()):
    print('%-2d %-30s %-15.6g %-15.6g'%(iax,ax.name,ax.lo_bound,ax.hi_bound))

## Create optimizer, define problem space and run optimization

1. Choose optimizer akgorithm : LBFGS, which makes use of analytic gradient
2. Set maximum verbosity which shows all evaluations of cost function
3. Set initial point for each parameter
4. Set low and high bounds on parameter space. Best results with smallest space!
5. Run optimizer
6. Print staus, coordinates of solution found and cost function there
7. Calculate error matrix and print errors (from diagnonals of matrix)

In [ ]:
opt_g = calin.math.optimizer.NLOptOptimizer("LD_LBFGS", like_g)
opt_g.set_verbosity_level(calin.math.optimizer.OptimizerVerbosityLevel_MAX);
opt_g.set_abs_tolerance(0.0001);
opt_g.set_initial_values([0.5, 30.0, 0.5,  8.0,  0.5]);
opt_g.set_limits_lo([0.01,  20.0,  0.25,   4.0,  0.25])
opt_g.set_limits_hi([3.0,   50.0,  2.00,  20.0,  5.0])
status, xopt_g, fval_g = opt_g.minimize()
print(status, xopt_g, fval_g)
status, err_mat_g = opt_g.calc_error_matrix()
xerr_g = sqrt(err_mat_g.diagonal())
print(xerr_g)

## Plot solution over the data

In [ ]:
calin.plotting.plot_histogram(mes_hist)
xlabel('Signal [DC]')
ylabel('Events per %d DC bin'%mes_hist.dxval())

ihist = range(0,mes_hist.nbin());
xhist = mes_hist.all_xval_center()

mes_model_g.set_parameter_values(xopt_g)
ymodel_g = \
    list(map(lambda x: mes_hist.sum_w()*mes_hist.dxval()*mes_model_g.pdf_mes(x),xhist))
plot(xhist,ymodel_g,lw=1.5, label='Gaussian')

ax = gca()
#ax.set_yscale('log')

## Plot the Single electron spectrum

In [ ]:
ses_x = arange(0,20,0.1);
ses_y = fromiter(map(ses_g_pdf.value_1d, ses_x),dtype='float')
plot(ses_x, ses_y)
#axis([0,250,0,0.02])
xlabel('Signal [ADC]')
ylabel('Probability density [1/ADC]')
#gcf().savefig('icrr_ses.pdf')

## Calculate the PMT gain, resolution and ENF

In [ ]:
qscale = 1
gunits = "ADC"
ses_norm = sum(ses_y*qscale)
ses_mean = sum(ses_y*ses_x*qscale)/ses_norm
ses_rms = sqrt(sum(ses_y*ses_x**2*qscale)/ses_norm - ses_mean**2)
print("Norm, mean, RMS [1,%s,%s]: "%(gunits,gunits),ses_norm, ses_mean, ses_rms)
print("Gain [%s]: "%gunits,ses_mean)
print("Light intensity [PE/pulse]: ",xopt_g[0])
print("Resolution: ",ses_rms/ses_mean)
print("ENF: ",sqrt(1+(ses_rms/ses_mean)**2))

## Use "robust" MLE cost function (experimental!)

This section shows how deweighting points in the the tails of the model can improve the fit. It should be considered experimental. The basic approach is to modify the log(probability) for each bin that is summed in the likelihood using a function rho which is linear at low values becoming asymptotically constant. This is a cost function that comes under the umbrella of being an M-Estimate, an it effectively "Winsorizes" the data.

More description is needed!

In [ ]:
rho = calin.math.data_modeling.ModifiedHyperbolicLikelihoodRhoFunction(7.0,2.0)
#like_mest = calin.calib.spe_fit.SPERobust(mes_model_g, mes_hist, rho) # ped_hist, rho)
like_mest = calin.calib.spe_fit.SPERobust(mes_model_g, mes_hist, ped_hist, rho)

In [ ]:
opt_mest = calin.math.optimizer.NLOptOptimizer("LD_LBFGS", like_mest)
opt_mest.set_verbosity_level(calin.math.optimizer.OptimizerVerbosityLevel_MAX);
opt_mest.set_abs_tolerance(0.0001);
opt_mest.set_initial_values(xopt_g);
opt_mest.set_limits_lo([0.5,  20.0,  0.5,    3.0,  0.5])
opt_mest.set_limits_hi([3.0,  50.0,  2.0,   15.0,  3.0])
status, xopt_mest, fval_mest = opt_mest.minimize()
print(status, xopt_mest, fval_mest)
status, err_mat_mest = opt_mest.calc_error_matrix()
xerr_mest = sqrt(err_mat_mest.diagonal())
print(xerr_mest)

In [ ]:
calin.plotting.plot_histogram(mes_hist)
xlabel('Signal [DC]')
ylabel('Events per %d DC bin'%mes_hist.dxval())

ihist = range(0,mes_hist.nbin());
xhist = mes_hist.all_xval_center()

mes_model_g.set_parameter_values(xopt_mest)
ymodel_mest = \
    list(map(lambda x: mes_hist.sum_w()*mes_hist.dxval()*mes_model_g.pdf_mes(x),xhist))
plot(xhist,ymodel_mest,lw=1.5, label='Gaussian')

ax = gca()
#ax.set_yscale('log')

## Calculate the PMT gain, resolution and ENF

In [ ]:
ses_x = arange(0,20,0.1);
ses_y = fromiter(map(ses_g_pdf.value_1d, ses_x),dtype='float')

qscale = 1
gunits = "ADC"
ses_norm = sum(ses_y*qscale)
ses_mean = sum(ses_y*ses_x*qscale)/ses_norm
ses_rms = sqrt(sum(ses_y*ses_x**2*qscale)/ses_norm - ses_mean**2)
print("Norm, mean, RMS [1,%s,%s]: "%(gunits,gunits),ses_norm, ses_mean, ses_rms)
print("Gain [%s]: "%gunits,ses_mean)
print("Light intensity [PE/pulse]: ",xopt_g[0])
print("Resolution: ",ses_rms/ses_mean)
print("ENF: ",sqrt(1+(ses_rms/ses_mean)**2))